In [1]:
import os
os.path.getsize('D:\DataGlacier Internship\Week6\Dataset\Trip_Records.csv')

2604553410

In [2]:
import pandas as pd
import time
file_path = 'D:\DataGlacier Internship\Week6\Dataset\Trip_Records.csv'

# Method 1: Pandas
start_time_pandas = time.time()
df_pandas = pd.read_csv(file_path)
end_time = time.time()
time_pandas = end_time - start_time_pandas
print("Time taken by Pandas:", time_pandas, "seconds")

C:\Users\varsh\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Time taken by Pandas: 548.6511058807373 seconds


In [3]:
from dask import dataframe as dd

# Method 2: Dask
start_time_desk = time.time()
df_dask = dd.read_csv(file_path,dtype={'DOLocationID': 'float64','PULocationID': 'float64','trip_time': 'float64'})
end_time = time.time()
time_dask = end_time - start_time_desk
print("Time taken by Dask:", time_dask, "seconds")

Time taken by Dask: 3.003405809402466 seconds


In [4]:
def clean_col_names(df):
    df.columns = df.columns.str.replace('[^a-zA-Z0-9]', '', regex=True).str.strip().str.lower()
    return df

data = df_dask.map_partitions(clean_col_names)

In [5]:
data.head()

,hvfhslicensenum,dispatchingbasenum,originatingbasenum,requestdatetime,onscenedatetime,pickupdatetime,dropoffdatetime,pulocationid,dolocationid,tripmiles,...,salestax,congestionsurcharge,airportfee,tips,driverpay,sharedrequestflag,sharedmatchflag,accessarideflag,wavrequestflag,wavmatchflag
0,HV0005,B03406,NaN,03/04/2022 07:52:25 PM,NaN,03/04/2022 07:55:46 PM,03/04/2022 08:12:33 PM,87.0,45.0,1.678,...,1.39,2.75,0.0,3.03,10.89,N,N,N,N,N
1,HV0005,B03406,NaN,03/04/2022 07:35:00 PM,NaN,03/04/2022 07:37:16 PM,03/04/2022 08:01:47 PM,112.0,141.0,5.315,...,1.92,2.75,0.0,0.00,19.24,N,N,N,N,N
2,HV0005,B03406,NaN,03/04/2022 07:27:57 PM,NaN,03/04/2022 07:34:05 PM,03/04/2022 08:02:38 PM,138.0,148.0,10.901,...,3.42,2.75,2.5,0.00,27.88,N,N,N,N,N
3,HV0003,B03404,B03404,03/04/2022 07:14:10 PM,03/04/2022 07:20:45 PM,03/04/2022 07:21:17 PM,03/04/2022 07:38:43 PM,97.0,61.0,2.350,...,1.34,0.00,0.0,0.00,12.99,N,N,NaN,N,N
4,HV0003,B03404,B03404,03/04/2022 07:06:41 PM,03/04/2022 07:09:34 PM,03/04/2022 07:10:46 PM,03/04/2022 07:33:09 PM,114.0,262.0,7.020,...,2.20,2.75,0.0,4.57,23.99,N,N,NaN,N,N


In [6]:
data.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 24 entries, hvfhslicensenum to wavmatchflag
dtypes: object(12), float64(12)

In [7]:
print('Number of rows : ',len(data.index))

Number of rows :  16049320


In [8]:
print('Number of columns : ',len(data.columns))

Number of columns :  24


In [9]:
data_cols = data.columns
data_cols

Index(['hvfhslicensenum', 'dispatchingbasenum', 'originatingbasenum',
       'requestdatetime', 'onscenedatetime', 'pickupdatetime',
       'dropoffdatetime', 'pulocationid', 'dolocationid', 'tripmiles',
       'triptime', 'basepassengerfare', 'tolls', 'bcf', 'salestax',
       'congestionsurcharge', 'airportfee', 'tips', 'driverpay',
       'sharedrequestflag', 'sharedmatchflag', 'accessarideflag',
       'wavrequestflag', 'wavmatchflag'],
      dtype='object')

In [10]:
# remove special characters from column names
df_pandas.columns = df_pandas.columns.str.replace('[^a-zA-Z0-9]+', '')

# remove whitespaces from column names
df_pandas.columns = df_pandas.columns.str.strip().str.lower()

# check for missing values in each column
missing_values = df_pandas.isnull().sum()

# check for duplicates
duplicates = df_pandas[df_pandas.duplicated()]

In [11]:
df_pandas.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'pulocationid', 'dolocationid', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

In [13]:
#We will load this YAML and validate the columns
import yaml

with open('D:\DataGlacier Internship\Week6\schema.yaml', 'r') as file:
    schema = yaml.safe_load(file)

# Validate columns
assert set(df_pandas.columns) == set(schema['columns']), "Column mismatch!"

In [31]:
#After processing, write the data to a pipe-separated, gzipped text file
subset = df_pandas[:1000000]
subset.to_csv('D:\DataGlacier Internship\Week6\output_file.gz', sep='|', compression='gzip', index=False)

In [32]:
os.path.getsize('D:\DataGlacier Internship\Week6\output_file.gz')

33073582